In [1]:
# =============================
# 🔧 CONFIGURATION
# =============================

# Choose LLM Provider: "openai" or "azure"
LLM_PROVIDER = "azure"  # Change to "azure" for Azure OpenAI

# =============================
# 🗝️ API Keys & Endpoints
# =============================

# OpenAI Settings
OPENAI_API_KEY = "<Your OpenAI Key Here>"  # <-- Set your OpenAI API Key here or leave blank for .env

# Azure OpenAI Settings
AZURE_API_KEY = "<Your Azure OpenAI Key Here>"  # <-- Set your Azure API Key here or leave blank for .env
AZURE_ENDPOINT = "<Your Azure OpenAI Endpoint Here>"  # <-- Your Azure endpoint

# =============================
# 📦 Model and Deployment
# =============================

# OpenAI model
OPENAI_MODEL = "gpt-4o"

# Azure OpenAI deployment & API version
AZURE_MODEL = "gpt-4o"
AZURE_DEPLOYMENT = "gpt-4o"  # <-- Azure deployment name
AZURE_API_VERSION = "2024-10-21"  # Example API version

# Temperature setting (controls creativity)
TEMPERATURE = 0.7

In [2]:
# =============================
# 📦 IMPORTS & ENV LOADER
# =============================
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Fallback to .env values if not set in config
OPENAI_API_KEY = OPENAI_API_KEY or os.getenv("OPENAI_API_KEY")

AZURE_API_KEY = AZURE_API_KEY or os.getenv("AZURE_OPENAI_API_KEY")
AZURE_ENDPOINT = AZURE_ENDPOINT or os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_API_VERSION = AZURE_API_VERSION or os.getenv("AZURE_OPENAI_API_VERSION", "2024-07-01-preview")
AZURE_DEPLOYMENT = AZURE_DEPLOYMENT or os.getenv("AZURE_OPENAI_DEPLOYMENT")

# =============================
# ✅ VALIDATION
# =============================
def validate_keys_and_endpoints():
    print("🔑 LLM Configuration Check:")
    if LLM_PROVIDER == "openai":
        if OPENAI_API_KEY:
            print(f"✅ OpenAI API Details: FOUND")
        else:
            print("❌ OpenAI API Key or Endpoint MISSING!")
    elif LLM_PROVIDER == "azure":
        if AZURE_API_KEY and AZURE_ENDPOINT and AZURE_API_VERSION and AZURE_DEPLOYMENT:
            print(f"✅ Azure API Details: FOUND")
        else:
            print("❌ Azure API Key, Endpoint, Deployment, or API Version MISSING!")
    else:
        print("❌ Invalid LLM_PROVIDER selected. Must be 'openai' or 'azure'.")

# Run validation
validate_keys_and_endpoints()

🔑 LLM Configuration Check:
✅ Azure API Details: FOUND


In [3]:
# =============================
# 🧠 LLMConnector CLASS
# =============================
from openai import OpenAI, AzureOpenAI

class LLMConnector:
    def __init__(self, provider=LLM_PROVIDER, model=None, temperature=TEMPERATURE):
        """
        provider: 'openai' or 'azure'
        model: Model name (e.g., gpt-4 or gpt-35-turbo)
        temperature: Sampling temperature for responses
        """
        self.provider = provider.lower()
        self.model = model
        self.temperature = temperature
        self.client = None
        self._configure_client()

    def _configure_client(self):
        if self.provider == "openai":
            self.model = OPENAI_MODEL
            if not OPENAI_API_KEY:
                raise ValueError("❌ OpenAI API Key is missing!")
            self.client = OpenAI(
                api_key=OPENAI_API_KEY
            )
            print(f"✅ Connected to OpenAI (model: {self.model or OPENAI_MODEL})")
        
        elif self.provider == "azure":
            if not AZURE_API_KEY or not AZURE_ENDPOINT or not AZURE_DEPLOYMENT:
                raise ValueError("❌ Azure OpenAI configuration missing!")
            self.client = AzureOpenAI(
                api_key=AZURE_API_KEY,
                azure_endpoint=AZURE_ENDPOINT,
                azure_deployment=AZURE_DEPLOYMENT,
                api_version=AZURE_API_VERSION
            )
            print(f"✅ Connected to Azure OpenAI (deployment: {AZURE_DEPLOYMENT})")
        
        else:
            raise ValueError("❌ Invalid provider. Use 'openai' or 'azure'.")

    def get_completion(self, prompt, max_tokens=None):
        """
        Generate a completion for a given prompt.
        """
        try:
            if self.provider == "openai":
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=self.temperature
                )
            elif self.provider == "azure":
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=self.temperature
                )
            return response.choices[0].message
        except Exception as e:
            print(f"❌ Error during completion: {e}")
            return None

In [4]:
# # =============================
# # 🚀 EXAMPLE USAGE
# # =============================

# # Toggle Debug Mode: Set to True to see raw JSON response
# DEBUG_MODE = False

# # 🔄 Create a connector instance (uses global LLM_PROVIDER)
# connector = LLMConnector()

# # 📝 Define your prompt
# prompt = "Explain the key differences between LLM training and inference in simple terms."

# # 📡 Get a response
# response = connector.get_completion(prompt)

# # 📋 Handle and display response
# if response:
#     if DEBUG_MODE:
#         # 🛠️ Print full raw response JSON
#         print("🔎 Raw API Response:")
#         from pprint import pprint
#         pprint(response)
#     else:
#         # 💬 Print only the response text
#         print("💬 LLM Response:")
#         print(response['content'] if isinstance(response, dict) else response)
# else:
#     print("⚠️ No response received.")